In [1]:
import pandas as pd
import os

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

from tkinter import messagebox

import re
import time

In [2]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath1 = filedialog.askopenfilename()

正在打开……


### 读取源数据

In [3]:
df1=pd.read_excel(datapath1,sheet_name='终端购机',dtype='str')
df2=pd.read_excel(datapath1,sheet_name='无宽客户',dtype='str')
df3=pd.read_excel(datapath1,sheet_name='全家享2.0',dtype='str')

### 备份

In [4]:
df1_bck=df1.copy()
df2_bck=df2.copy()
df3_bck=df3.copy()

### 还原

In [27]:
df1=df1_bck
df2=df2_bck
df3=df3_bck

### 读取派单规则

In [5]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath_r = filedialog.askopenfilename()

正在打开……


In [6]:
df_r1=pd.read_excel(datapath_r,sheet_name='渠道')
df_r2=pd.read_excel(datapath_r,sheet_name='直销')
df_r3=pd.read_excel(datapath_r,sheet_name='三大家')


### 数据清理

将不是数字等转化为数字。

In [28]:
# df1[['服务中心ID','网格ID']] = df1[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df2[['服务中心ID','网格ID']] = df2[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df3[['服务中心ID','网格ID']] = df3[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
df1 = df1.apply(pd.to_numeric, errors='coerce').fillna(0.0)
df2 = df2.apply(pd.to_numeric, errors='coerce').fillna(0.0)
df3 = df3.apply(pd.to_numeric, errors='coerce').fillna(0.0)

### 递归，根据各种不同条件，取满足条件的号码，并依次遍历

In [26]:
#取出各条件号码，依次遍历

def handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin):
    # print(len(df_t_tmp1))
    # 结束条件
    if (len(df_t_tmp1) == 10000):
        return df_res_total

    if ((len(df_t_tmp1) >= pdl1)):
        df_res = df_t_tmp1.head(pdl1).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        return df_res_total
    else:
        last_pdl = pdl1 - len(df_t_tmp1)
        df_res = df_t_tmp1.head(len(df_t_tmp1)).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        df_t_tmp1 = stack.pop()  # stack.pop()也可
        return handle(df_t_tmp1, last_pdl, row, stack, df_res_total, df_res,df_orgin)

### 渠道

无宽客户

In [29]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r1.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()
    df_t_res6=pd.DataFrame()
    df_t_res7=pd.DataFrame()
    df_t_res8=pd.DataFrame()
    df_t_res9=pd.DataFrame()
    df_t_res10=pd.DataFrame()
    df_tmp=df2[df2['网格ID']==row['微网格ID']]
    df_t_tmp1=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==1)]
    df_t_tmp2=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==1)]
    df_t_tmp3=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==0)]
    df_t_tmp4=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==1)]
    df_t_tmp5=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==0)&(df_tmp['不限量']==1)]
    df_t_tmp6=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==1)]
    df_t_tmp7=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==1)]
    df_t_tmp8=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==0)]
    df_t_tmp9=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==1)]
    df_t_tmp10=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==0)&(df_tmp['不限量']==1)]

    df_orgin=df2#修改条件

    pdl1 = row['无宽客户']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp10)
    stack.append(df_t_tmp9)
    stack.append(df_t_tmp8)
    stack.append(df_t_tmp7)
    stack.append(df_t_tmp6)
    stack.append(df_t_tmp5)
    stack.append(df_t_tmp4)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
                     
df_res_total_l.to_excel('渠道-无宽客户-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享2.0

In [30]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r1.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()

    df_tmp=df3[df3['网格ID']==row['微网格ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否质差客户']==0)&(df_tmp['是否裸奔高价值（存50送120）']==1)]


    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('渠道-全家享2.0-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]


### 直销

无宽客户

In [31]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r2.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()
    df_t_res6=pd.DataFrame()
    df_t_res7=pd.DataFrame()
    df_t_res8=pd.DataFrame()
    df_t_res9=pd.DataFrame()
    df_t_res10=pd.DataFrame()
    df_tmp=df2[df2['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==1)]
    df_t_tmp2=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==1)]
    df_t_tmp3=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==0)]
    df_t_tmp4=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==1)]
    df_t_tmp5=df_tmp[(df_tmp['机龄']>=24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==0)&(df_tmp['不限量']==1)]
    df_t_tmp6=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==1)]
    df_t_tmp7=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==1)]
    df_t_tmp8=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==0)]
    df_t_tmp9=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==1)]
    df_t_tmp10=df_tmp[(df_tmp['机龄']<24)&(df_tmp['个人高价值']==0)&(df_tmp['我号异宽']==0)&(df_tmp['5G资费']==1)&(df_tmp['个人修复']==0)&(df_tmp['不限量']==1)]


    df_orgin=df2#修改条件

    pdl1 = row['无宽客户']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp10)
    stack.append(df_t_tmp9)
    stack.append(df_t_tmp8)
    stack.append(df_t_tmp7)
    stack.append(df_t_tmp6)
    stack.append(df_t_tmp5)
    stack.append(df_t_tmp4)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
                     
df_res_total_l.to_excel('直销-无宽客户-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享

In [32]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r2.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()

    df_tmp=df3[df3['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否质差客户']==0)&(df_tmp['是否裸奔高价值（存50送120）']==1)]

    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('直销-全家享2.0-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

三大家


终端购机

In [33]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r3.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()

    df_tmp=df1[df1['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否为宽带客户']==0)]
    df_t_tmp2=df_tmp[(df_tmp['是否为宽带客户']==1)]


    df_orgin=df1#修改条件

    pdl1 = row['终端购机']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('三大家-终端购机-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享

In [34]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r3.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_t_res1=pd.DataFrame()
    df_t_res2=pd.DataFrame()
    df_t_res3=pd.DataFrame()
    df_t_res4=pd.DataFrame()
    df_t_res5=pd.DataFrame()

    df_tmp=df3[df3['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否质差客户']==1)&(df_tmp['是否裸奔高价值（存50送120）']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否质差客户']==0)&(df_tmp['是否裸奔高价值（存50送120）']==1)]

    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('三大家-全家享-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]